In [2]:
import pickle
import time
from typing import Dict, Any, Optional
import copy

class Memento:
    """备忘录类 - 存储状态快照"""
    
    def __init__(self, state: Any, timestamp: float, description: str):
        self._state = copy.deepcopy(state)  # 深拷贝状态
        self._timestamp = timestamp
        self._description = description
        self._size = self._calculate_size()
    
    def get_state(self):
        """获取状态（返回深拷贝）"""
        return copy.deepcopy(self._state)
    
    def get_timestamp(self) -> float:
        """获取时间戳"""
        return self._timestamp
    
    def get_description(self) -> str:
        """获取描述"""
        return self._description
    
    def get_size(self) -> int:
        """获取快照大小（字节）"""
        return self._size
    
    def _calculate_size(self) -> int:
        """计算快照数据大小"""
        try:
            return len(pickle.dumps(self._state))
        except:
            return 0

class CADModel:
    """CAD 模型类 - 原发器（Originator）"""
    
    def __init__(self):
        self.shapes = {}
        self.next_id = 0
        self.properties = {
            'units': 'mm',
            'precision': 0.01,
            'author': 'CAD User'
        }
    
    def add_shape(self, shape_type: str, **kwargs) -> int:
        """添加几何体"""
        shape_id = self.next_id
        self.shapes[shape_id] = {
            'id': shape_id,
            'type': shape_type,
            'created_time': time.time(),
            **kwargs
        }
        self.next_id += 1
        print(f"✅ 添加 {shape_type} (ID: {shape_id})")
        return shape_id
    
    def remove_shape(self, shape_id: int) -> bool:
        """删除几何体"""
        if shape_id in self.shapes:
            del self.shapes[shape_id]
            print(f"🗑️ 删除几何体 (ID: {shape_id})")
            return True
        return False
    
    def modify_shape(self, shape_id: int, **kwargs):
        """修改几何体属性"""
        if shape_id in self.shapes:
            self.shapes[shape_id].update(kwargs)
            print(f"🔧 修改几何体 (ID: {shape_id})")
            return True
        return False
    
    def get_model_info(self) -> Dict:
        """获取模型信息"""
        return {
            'shape_count': len(self.shapes),
            'shape_types': [s['type'] for s in self.shapes.values()],
            'properties': self.properties
        }
    
    def create_memento(self, description: str = "") -> Memento:
        """创建快照"""
        state = {
            'shapes': self.shapes,
            'next_id': self.next_id,
            'properties': self.properties
        }
        return Memento(state, time.time(), description)
    
    def restore_from_memento(self, memento: Memento):
        """从快照恢复"""
        state = memento.get_state()
        self.shapes = state['shapes']
        self.next_id = state['next_id']
        self.properties = state['properties']
        print(f"🔄 恢复到快照: {memento.get_description()}")

class SnapshotManager:
    """快照管理器 - 负责人（Caretaker）"""
    
    def __init__(self, max_snapshots: int = 10):
        self.snapshots: List[Memento] = []
        self.current_index = -1
        self.max_snapshots = max_snapshots
        self.total_size = 0
    
    def save_snapshot(self, model: CADModel, description: str = ""):
        """保存快照"""
        # 如果不在历史末尾，删除后续快照
        if self.current_index < len(self.snapshots) - 1:
            removed_snapshots = self.snapshots[self.current_index + 1:]
            for snapshot in removed_snapshots:
                self.total_size -= snapshot.get_size()
            self.snapshots = self.snapshots[:self.current_index + 1]
        
        # 创建新快照
        memento = model.create_memento(description)
        self.snapshots.append(memento)
        self.current_index += 1
        self.total_size += memento.get_size()
        
        # 限制快照数量
        while len(self.snapshots) > self.max_snapshots:
            removed = self.snapshots.pop(0)
            self.total_size -= removed.get_size()
            self.current_index -= 1
        
        print(f"💾 保存快照: {description} ({memento.get_size()} 字节)")
        self._print_memory_usage()
    
    def undo(self, model: CADModel) -> bool:
        """撤销到上一个快照"""
        if self.current_index > 0:
            self.current_index -= 1
            snapshot = self.snapshots[self.current_index]
            model.restore_from_memento(snapshot)
            return True
        else:
            print("❌ 无可撤销的快照")
            return False
    
    def redo(self, model: CADModel) -> bool:
        """重做到下一个快照"""
        if self.current_index < len(self.snapshots) - 1:
            self.current_index += 1
            snapshot = self.snapshots[self.current_index]
            model.restore_from_memento(snapshot)
            return True
        else:
            print("❌ 无可重做的快照")
            return False
    
    def get_snapshot_history(self):
        """获取快照历史"""
        print(f"📚 快照历史 (当前位置: {self.current_index + 1}/{len(self.snapshots)}):")
        for i, snapshot in enumerate(self.snapshots):
            marker = "➤" if i == self.current_index else " "
            timestamp = time.strftime("%H:%M:%S", time.localtime(snapshot.get_timestamp()))
            size_kb = snapshot.get_size() / 1024
            print(f"  {marker} {i}: {snapshot.get_description()} [{timestamp}] ({size_kb:.1f}KB)")
    
    def _print_memory_usage(self):
        """打印内存使用情况"""
        total_mb = self.total_size / (1024 * 1024)
        print(f"📊 快照内存使用: {total_mb:.2f} MB ({len(self.snapshots)} 个快照)")

In [3]:
def snapshot_pattern_demo():
    """快照模式演示"""
    
    print("📷 快照模式 Undo/Redo 演示")
    print("=" * 50)
    
    # 创建模型和快照管理器
    model = CADModel()
    snapshot_manager = SnapshotManager(max_snapshots=5)
    
    # 初始快照
    snapshot_manager.save_snapshot(model, "初始状态")
    
    # 执行一系列操作
    print("\n📝 执行建模操作:")
    
    # 添加几何体
    box_id = model.add_shape('box', width=10, height=20, depth=30)
    snapshot_manager.save_snapshot(model, "添加立方体")
    
    sphere_id = model.add_shape('sphere', center=(0, 0, 0), radius=15)
    snapshot_manager.save_snapshot(model, "添加球体")
    
    cylinder_id = model.add_shape('cylinder', radius=8, height=25)
    snapshot_manager.save_snapshot(model, "添加圆柱体")
    
    # 修改几何体
    model.modify_shape(box_id, color='red', material='steel')
    snapshot_manager.save_snapshot(model, "修改立方体属性")
    
    print(f"\n当前模型信息: {model.get_model_info()}")
    
    # 撤销操作
    print("\n↩️ 撤销操作:")
    snapshot_manager.undo(model)  # 撤销属性修改
    snapshot_manager.undo(model)  # 撤销添加圆柱体
    
    print(f"撤销后模型信息: {model.get_model_info()}")
    
    # 重做操作
    print("\n↪️ 重做操作:")
    snapshot_manager.redo(model)   # 重做添加圆柱体
    
    print(f"重做后模型信息: {model.get_model_info()}")
    
    # 显示快照历史
    print("\n")
    snapshot_manager.get_snapshot_history()
    
    # 执行新操作（会清除后续快照）
    print("\n📝 执行新操作（将清除后续快照）:")
    model.add_shape('cone', radius=6, height=12)
    snapshot_manager.save_snapshot(model, "添加圆锥体")
    
    print("\n")
    snapshot_manager.get_snapshot_history()

# 运行演示
snapshot_pattern_demo()

📷 快照模式 Undo/Redo 演示
💾 保存快照: 初始状态 (110 字节)
📊 快照内存使用: 0.00 MB (1 个快照)

📝 执行建模操作:
✅ 添加 box (ID: 0)
💾 保存快照: 添加立方体 (192 字节)
📊 快照内存使用: 0.00 MB (2 个快照)
✅ 添加 sphere (ID: 1)
💾 保存快照: 添加球体 (253 字节)
📊 快照内存使用: 0.00 MB (3 个快照)
✅ 添加 cylinder (ID: 2)
💾 保存快照: 添加圆柱体 (295 字节)
📊 快照内存使用: 0.00 MB (4 个快照)
🔧 修改几何体 (ID: 0)
💾 保存快照: 修改立方体属性 (328 字节)
📊 快照内存使用: 0.00 MB (5 个快照)

当前模型信息: {'shape_count': 3, 'shape_types': ['box', 'sphere', 'cylinder'], 'properties': {'units': 'mm', 'precision': 0.01, 'author': 'CAD User'}}

↩️ 撤销操作:
🔄 恢复到快照: 添加圆柱体
🔄 恢复到快照: 添加球体
撤销后模型信息: {'shape_count': 2, 'shape_types': ['box', 'sphere'], 'properties': {'units': 'mm', 'precision': 0.01, 'author': 'CAD User'}}

↪️ 重做操作:
🔄 恢复到快照: 添加圆柱体
重做后模型信息: {'shape_count': 3, 'shape_types': ['box', 'sphere', 'cylinder'], 'properties': {'units': 'mm', 'precision': 0.01, 'author': 'CAD User'}}


📚 快照历史 (当前位置: 4/5):
    0: 初始状态 [17:45:48] (0.1KB)
    1: 添加立方体 [17:45:48] (0.2KB)
    2: 添加球体 [17:45:48] (0.2KB)
  ➤ 3: 添加圆柱体 [17:45:48] (0.3KB)
    4: 修改立方